# Euler 004

Euler 004 asks to find the largest palindromic number that can be created from
the product of two `4` digit numbers. This ipynb attempts to optimize the pursuit
of a more general question of "what is the largest palindromic number that can
be created from the product of two `n` digit number?"

## Import Block (and helper functions)

In [1]:
import math
import time

import numpy as np

In [2]:
# A function that provides the largest n digit number
#  and the smallest n digit number
def ndigit_max_min(n):
    largest = 10**n - 1
    smallest = 10**(n-1)
    return largest, smallest

# A function that checks if a provided number is a palindrome
def check_palind(x):
    def rev(num):
        ret = 0
        while num > 0:
            ret = ret*10 + num % 10
            num = num//10
        return ret
    y = rev(x)
    return x == y

def timing_report(test_function, no_trials, *args):
    times = np.zeros((no_trials))
    for i in range(no_trials):
        x = time.time()
        test_function(*args)
        y = time.time()
        times[i] = y-x
    print(f"Timing report:")
    print(f"mean: {np.mean(times)}, std-dev: {np.std(times)}")
    print(f"span: {np.max(times)-np.min(times)}")
    

## Constant memory solutions

### The worst "reasonable" solution
Please do not loop over all possible numbers by computing the largest possible product
(`(10**n-1)**2`) and the smallest possible product (`(10**(n-1))**2`) and looping downwards,
stopping when you find the first palindrome that has two `n` digit factors. It has one massive
benefit of early stopping, but you will also loop over (and attempt to factorize) a significant
quantity of numbers that aren't factorable to two `n` digit numbers.  

However, we can learn from this solution that we should loop downwards! Especially if we can
find a way to stop early.

In [7]:
# No, I will not be attempting to implement and optimize this potential algorithm

### A slightly better solution
Now we've learned we should loop only over the possible `n` digit pairings and that we should
loop backwards, so that we encounter larger answers first. With such knowledge in hand, we might
code something similar to something below. We can even add the subtle optimization of checking to
see if the current product we're checking is larger
than the largest palindrome we've found in order to short circuit our conditional and avoid computing
if a number is a palindrome if the number isn't even large enough to be the largest palindrome
in our search space.

In [4]:
def palindrome(n):
    largest = 0
    big, small = ndigit_max_min(n)
    for i in range(big, small, -1):
        for j in range(big, i-1, -1):
            # subtle optimization of short circuiting
            if (i*j > largest and check_palind(i*j)):
                largest = i*j
    return largest

timing_report(palindrome, 10, 4)

Timing report:
mean: 2.2628547668457033, std-dev: 0.029659305982784028
span: 0.09248685836791992


However, this looping scheme isn't as good as it could be. We often find ourselves checking smaller
values before we look at larger ones. For example, if `n=2`, our loop checks `97*97` before it
checks `96*99` even though the magnitude of the latter is larger than that of the former.
This means that somewhere, we can reduce the number of `n` digit pairs that we check to be 
palindromes. A better scheme for searching the `n` digit pair space is to go diagonally across
the pairing instead of straight up and down. What is kept consistent by the inner loop is the 
sum of the `n` digit pair. 

In [25]:
def palindrome(n):
    largest = 0
    big, small = ndigit_max_min(n)
    for i in range(2*big, 2*small, -1):
        mid = i/2
        upper = math.ceil(mid)
        lower = math.floor(mid)
        while (upper <= big and lower >= small) and (upper * lower > largest):
            if check_palind(upper*lower):
                largest = upper*lower
            upper += 1
            lower -= 1
    return largest

timing_report(palindrome, 10, 6)

Timing report:
mean: 0.8069505214691162, std-dev: 0.0034282688647126985
span: 0.012799263000488281


This provides a surprisingly ridiculous speed up, especially as `n` increases

Through all this, you might be thinking, why haven't we added an early stopping condition? That is
because both of these traversal methods fail to guarantee that our next iteration's product will
be smaller than the current iteration. Instead we must do our best to cut out work, using conditionals,
which is exactly what this specific traversal method gives us access to.

Now that we are traversing in a manner that has a mathematical constraing of the sum being held
constant in the inner loop and decreasing in the outer loop, we can stop early if the square root
of the largest number is larger than the midpoint our inner loop is searching around. This may eventually
give diminishing returns by the nature of the square root (it will exclude a smaller proportion
of the search space as `n` increases), but is quite a speed up in the lower digit length searches,
practically halving the average time when looking for multiplication by `6` digit integers.

In [26]:
def palindrome(n):
    largest = 0
    big, small = ndigit_max_min(n)
    for i in range(2*big, 2*small, -1):
        mid = i/2
        upper = math.ceil(mid)
        lower = math.floor(mid)
        while (upper <= big and lower >= small) and (upper * lower > largest):
            if check_palind(upper*lower):
                largest = upper*lower
            upper += 1
            lower -= 1
        if mid * mid <= largest:
            break
    return largest

timing_report(palindrome, 10, 6)

Timing report:
mean: 0.4055248022079468, std-dev: 0.00867037011112612
span: 0.03074502944946289


### Using some numeric properties of palindromes.

One of the numeric properties of palindromes is that if they have an even number of digits, the
palindrome must be divisible by 11. We know that a significant quantity of our search space is
made up of numbers with an even quantity of digits. However, in our search space we are not
guaranteed that the first palindro will be found in that space. For example, when looking at `n=3`,
we know that `999*999` gives us a six digit number, but at some point, we may begin to encounter
five digit numbers, so we can reduce the search space by forcing one of the two `n` digit numbers
to be divisible by `11` when the number of digits in the product is even.

In [27]:

def nonevenLength(n):
    return (math.floor(math.log10(n)+1) % 2 == 1)


def palindrome(n):

    start, end = ndigit_max_min(n)
    largest = [0, None]
    for i in range(2*start, 2*end, -1):
        mid = i/2
        upper = math.ceil(mid)
        lower = math.floor(mid)
        while (upper <= start and lower >= end) and (upper * lower > largest[0]):
            # a palindrome with odd number of digits doesn't need to be a multiple of 10
            prod = upper*lower
            if (upper % 11 == 0 or lower % 11 == 0 or nonevenLength(prod)) and check_palind(prod):
                largest[0] = prod
                largest[1] = (upper, lower)
            upper += 1
            lower -= 1
        if mid * mid <= largest[0]:
            break
    return largest


timing_report(palindrome, 50, 6)


Timing report:
mean: 0.16540193557739258, std-dev: 0.001650449181253839
span: 0.008331537246704102


Thanks for reading!